# Create murals from sample .json

To experiment with searching in murals, you first need some murals to search.

This notebook demonstrates how to use the MURAL API to create murals containing widgets defined in a .json file.

A sample .json file containing the contents of four sample murals is available here: [Sample .json file](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/Discovery_sample-murals.json)

The following image shows what the sample murals look like:

<img src="https://github.com/spackows/MURAL-API-Samples/blob/main/images/Discovery_sample-murals-to-search-horizontal.png?raw=true" width="95%" title="Sample murals" />

# Table of contents

<ol style="list-style: none; margin: 20px 0px 0px 0px; padding: 0px">
<li style="margin: 0px 0px 3px 0px;"><b>Step 1:</b> Read sample .json file</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 2:</b> Set your MURAL workspace ID</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 3:</b> Set your MURAL Oauth token</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 4:</b> Create sample rooms</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 5:</b> Create sample folders</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 6:</b> Create sample murals</li>
<li style="margin: 0px 0px 3px 0px;"><b>Step 7:</b> Populate murals with sample widgets</li>
</ol>

## Step 1: Read sample .json file

In [ ]:
import urllib
import json

url = "https://github.com/spackows/MURAL-API-Samples/raw/main/murals/Discovery_sample-murals.json"
response = urllib.request.urlopen( url )
encoding = response.info().get_content_charset( "utf8" )
g_murals_arr = json.loads( response.read().decode( encoding ) )

In [ ]:
print( json.dumps( g_murals_arr, indent = 3 ) )

## Step 2: Set your MURAL workspace ID

We need the ID of the workspace in which to create murals for searching.

You can get the workspace ID from your browser URL.

When you are in the Home page of your workspace, the URL in your browser looks something like this:
```
https://app.mural.co/t/<workspace_id>/home
```

What you need to pass to the MURAL API is just after the `/t/` and before the `/home`.

For example, if you have a workspace with this url:
```
https://app.mural.co/t/teamideas1234/home
```

Then, the workspace ID is: teamideas1234

In [ ]:
g_workspace_id = ""

## Step 3: Set your MURAL Oauth token

You must get an OAuth token by using OAuth 2.0.  For example, see: [Setting up OAuth 2.0 in Postman](https://developers.mural.co/public/docs/testing-with-postman#setting-up-oauth-20-in-postman)

In [ ]:
g_auth_token = ""

## Step 4: Create sample rooms

In [ ]:
import requests
import json
import re

def createRoom( auth_token, workspace_id, room_name ):
    # https://developers.mural.co/public/reference/createroom
    url = "https://app.mural.co/api/public/v1/rooms"
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "workspaceId" : workspace_id, 
              "name"        : room_name,
              "type"        : "open" }
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    if "id" not in response_json["value"]:
        print( "No id returned" )
        return None
    return response_json["value"]["id"]

In [ ]:
rooms_created = {}
for mural in g_murals_arr:
    room_name = mural["room"]["name"]
    if room_name in rooms_created.keys():
        mural["room"]["id"] = rooms_created[ room_name ]
        continue
    room_id = createRoom( g_auth_token, g_workspace_id, room_name )
    rooms_created[ room_name ] = room_id
    mural["room"]["id"] = room_id

print( json.dumps( g_murals_arr, indent=3 ) )

## Step 5: Create sample folders

In [ ]:
def createFolder( auth_token, room_id, folder_name ):
    # https://developers.mural.co/public/reference/createroomfolder
    url = "https://app.mural.co/api/public/v1/rooms/" + str( room_id ) + "/folders"
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "name" : folder_name }
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    if "id" not in response_json["value"]:
        print( "No id returned" )
        return None
    return response_json["value"]["id"]

In [ ]:
folders_created = {}
for mural in g_murals_arr:
    room_id = mural["room"]["id"]
    folder_name = mural["folder"]["name"]
    if "" == folder_name:
        continue
    if folder_name in folders_created.keys():
        mural["folder"]["id"] = folders_created[ folder_name ]
        continue
    folder_id = createFolder( g_auth_token, room_id, folder_name )
    folders_created[ folder_name ] = folder_id
    mural["folder"]["id"] = folder_id

print( json.dumps( g_murals_arr, indent=3 ) )

## Step 6: Create sample murals

In [ ]:
def createMural( auth_token, workspace_id, room_id, folder_id, title ):
    # https://developers.mural.co/public/reference/createmural
    url = "https://app.mural.co/api/public/v1/murals"
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = { "workspaceId" : workspace_id, 
              "roomId"      : int( room_id ),  # If this is not a number, call fails
              "folderId"    : folder_id,
              "title"       : title }
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return None
    if "value" not in response_json:
        print( "No value returned" )
        return None
    if "id" not in response_json["value"]:
        print( "No id returned" )
        return None
    return response_json["value"]["id"]

In [ ]:
for mural in g_murals_arr:
    room_id = mural["room"]["id"]
    folder_id = mural["folder"]["id"]
    mural_name = mural["mural_name"]
    mural_id = createMural( g_auth_token, g_workspace_id, room_id, folder_id, mural_name )
    mural["id"] = mural_id

print( json.dumps( g_murals_arr, indent=3 ) )

## Step 7: Populate murals with sample widgets

In [ ]:
import copy

widget_types = {
    "sticky note" : "sticky-note",
    "shape"       : "shape",
    "text"        : "textbox"
}

def putWidget( auth_token, mural_id, widget ):
    # https://developers.mural.co/public/reference/createstickynote
    # https://developers.mural.co/public/reference/createshapewidget
    # https://developers.mural.co/public/reference/createtextbox
    # 
    widget_type = widget_types[ widget["type"] ]
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets/" + widget_type
    headers = { "Accept"        : "application/json", 
                "Content-Type"  : "application/json", 
                "Authorization" : "Bearer " + auth_token }
    parms = copy.deepcopy( widget )
    if "type" in parms:
        del parms["type"]
    response = requests.request( "POST", url, headers = headers, json = parms )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( "[ mural ID: " + mural_id + " ] [ " + widget["text"][:15] + " ]\nError: " + msg + "\n" + json.dumps( parms, indent=3 ) )
    else:
        print( "[ mural ID: " + mural_id + " ] [ " + widget["text"][:15] + " ] Success" )

In [ ]:
for mural in g_murals_arr:
    print( "\n\n" + mural["mural_name"] )
    mural_id = mural["id"]
    widgets_arr = mural["widgets_arr"]
    for widget in widgets_arr:
        putWidget( g_auth_token, mural_id, widget )